In [1]:
from modules.models import SchrodingerPinn, create_dense_model
from modules.data import simulate_schrodinger
from modules.plots import plot_schrodinger_model, plot_training_loss_linlog, plot_pointwise_error_mesh

2022-12-28 19:35:01.847977: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from modules.utils import load_mat_data

In [3]:
def f_init(tx):
    h = 2.0 / tf.cosh(tx[:, 1:2])
    return tf.concat([h, tf.zeros_like(h)], axis=1)

In [4]:
nls_data = load_mat_data("data/NLS.mat")
h_exact = nls_data['uu'].T.flatten()[:, None]
u_exact = np.real(h_exact)
v_exact = np.imag(h_exact)
t_exact = nls_data['tt'].reshape(-1, 1)
x_exact = nls_data['x'].reshape(-1, 1)
X, T = np.meshgrid(x_exact,t_exact) 
X_star = np.hstack((T.flatten()[:,None], X.flatten()[:,None]))
tx_exact = tf.convert_to_tensor(X_star, dtype=tf.float32)
h_exact = np.concatenate([u_exact, v_exact], axis=1)
h_exact = tf.convert_to_tensor(h_exact, dtype=tf.float32)

2022-12-28 19:35:10.609491: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-28 19:35:11.146791: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-28 19:35:11.147516: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-28 19:35:11.148643: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [5]:
#randomly pick n_samples samples from the tx_exact
n_samples = 5000
idx = np.random.choice(tx_exact.shape[0], n_samples, replace=False)
tx_train = tf.gather(tx_exact, idx)
h_train = tf.gather(h_exact, idx)

In [6]:
x_start = -5
length = 10
time = np.pi / 2
(_, residuals_exact), (tx_init, h_init_exact), tx_bnd = simulate_schrodinger_v2(n_samples, f_init, x_start, length, time)

In [7]:
inputs = [tx_train, tx_init, tx_bnd]
outputs = [h_train, residuals_exact, h_init_exact]

In [8]:
backbone = create_dense_model([64]*3, activation='gelu', initializer='glorot_normal', \
    n_inputs=2, n_outputs=2)

In [9]:
pinn = SchrodingerPinn(backbone)

In [10]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-3,
    decay_steps=500,
    decay_rate=0.92,
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
pinn.compile(optimizer=optimizer)

In [11]:
history = pinn.fit(inputs, outputs, epochs=20000, batch_size=200, verbose=0)

In [ ]:
plot_training_loss_linlog(history.history)

In [ ]:
plot_schrodinger_model(pinn.backbone, x_start, length, time)